### Let's build a webpage!
write a tutorial describing how you constructed your webpage

link to the GitHub repository: 

### 1. Python Functions

**1.1 get_message_db( )**

The function get_message_db( ) handles creating the database of messages.

It first checks whether there is a database called message_db in the g attribute of the app. If not, then connect to that database, ensuring that the connection is an attribute of g. 

Then it checks whether a table called messages exists in message_db, and create it if not. It will give the table an id column (integer), a handle column (text), a message column (text), and return the connection g.message_db.

In [ ]:
def get_message_db():
    if 'message_db' not in g:
        g.message_db = sqlite3.connect('message_db.sqlite')

    cursor = g.message_db.cursor()
    cmd = """CREATE TABLE IF NOT EXISTS messages(id INTEGER, handle TEXT, message TEXT)"""
    cursor.execute(cmd)
    return g.message_db

**1.2 insert_message(request)**

The function insert_message(request) handles inserting a user message into the database of messages.

It first extracts the message and the handle from request. Meanwhile we need to ensure that the submit.html template creates these fields from user input by appropriately specifying the name of the input elements. 

Next, use a cursor to insert the message *(an ID number, the handle, and the message itself)* into the message database. Note: it is necessary to run db.commit() after inserting a row into db in order to ensure that the row insertion has been saved.

To ensure the ID number of each message is unique by setting the ID number of a message equal to one plus the current number of rows in message_db.

Close the database connection within the function.

In [2]:
def insert_message(request):
    handle = request.form['handle']
    message = request.form['message']
    
    db = get_message_db()
    cursor = db.cursor()
    cmd = """SELECT COUNT(*) FROM messages"""
    
    cursor.execute(cmd)
    db.execute(f""" INSERT INTO messages(id, handle, message) VALUES ({cursor.fetchone()[0]+1}, "{handle}", "{message}");""")
    db.commit()
    db.close()

**1.3 random_messages(n)**

The function random_messages(n) returns a collection of n random messages from the message_db, or fewer if necessary.

Close the database connection within the function.

In [ ]:
def random_messages(n):
    db = get_message_db()
    cursor = db.cursor()
    cmd = f"""SELECT * FROM messages LIMIT {n}"""
    
    cursor.execute(cmd)
    data = cursor.fetchall()
    db.close()
    return data

**1.4 main( )**

The main( ) function renders the main_better.html, which will be the main page when users first access the webpage.

In [3]:
@app.route("/")
def main():
    return render_template("main_better.html")

**1.5 submit( )**

The submit( ) function supports both the 'POST' and the 'GET' method. 

In the POST case, if it calls the function insert_message( ) successfully, it then renders the submit.html template and display a "Thank You For Your Submission!". Otherwise, it will display an error message "Submission Failed."

In the GET case, which is the default method, it just renders the submit.html template with no other parameters. 

In [ ]:
@app.route('/submit/', methods=['POST', 'GET'])
def submit():
    if request.method == 'POST':
        try:
            insert_message(request)
            return render_template('submit.html', submission = True)
        except:
            return render_template('submit.html', failure = True)
    return render_template('submit.html')

**1.6 view( )**

The view( ) function renders the view.html template and displays the messages extracted from random_messages(n).

In [4]:
def view():
    messages=random_messages(8)
    return render_template('view.html', messages=messages)

**1.7 viewall( )**

The viewall( ) function renders the viewall.html template and displays all the messages submitted to the webpage.

In [ ]:
def viewall():
    db = get_message_db()
    messages = db.execute("SELECT * FROM messages").fetchall()
    return render_template('viewall.html', messages = messages)

A discussion of at least one of the template files you used in your app. You can copy the source code of the template file into your markdown post.

### 2. Template Files

**Example: submit.html**

submit.html extends base.html, which contains the nagivation links to main, submit, view, and view all.

The block header contains the title of the page at the top.

The block content contains 3 features on the page, where "Username" allows the users to enter their names and "Message" allows the users to enter messages. The "submit" button checks check whether a submission is successful.

If submission is successful, the page will display a thank you message and if not, will give a failure message.

{% extends 'base.html' %}

{% block header %}
  <h1>{% block title %}Submit{% endblock %}</h1>
{% endblock %}

{% block content %}
  <form method="post" enctype="multipart/form-data">
    <br>
    <label for="name"><h10>Username:</h10> </label>
    <input type="text" name="handle" id="handle">
    <br>
    <br>
    <label for="message"><h10>Message:</h10></label>
    <input type="text" name="message" id="message">
    <br>
    <br>
    <input type="submit" value="Submit">
    <br>
  </form>

  {% if submission %}
    <br>
    Thank You For Your Submission!
  {% endif %}

  {% if failure %}
    <br>
    Submission Failed.
  {% endif %}

{% endblock %}

Your blog post must include two screencaps:

### 3. Screencaps

**3.1 A user submitting a message.**

**3.2 A user viewing submitted messages.** 

Additionally, please include in your blog post a link to the GitHub repository containing the code for your app.